##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Обучи свою первую нейросеть: простой классификатор

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Читай на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ru/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запусти в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ru/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучай код на GitHub</a>
  </td>
</table>

Это пошаговое руководство поможет тебе обучить свою первую нейросеть, которая будет классифицировать картинки предметов одежды.

Все в порядке если какие-то нюансы будет непонятны тебе в первый раз: это быстрая, ознакомительная программа по TensorFlow, которая показывает и объясняет новые детали по мере их появления.

В уроке используется [tf.keras](https://www.tensorflow.org/guide/keras), высокоуровневый Keras API для построения и обучения моделей в TensorFlow.

In [ ]:
# Импортируем TensorFlow и tf.keras
import tensorflow as tf
from tensorflow import keras

# А также добавим вспомогательные библиотеки для вычислений и вывода данных на экран
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Загружаем Fashion MNIST датасет

Мы будем использовать [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) датасет, который состоит из 70,000 черно-белых изображений в 10 категориях. На каждом изображении по одному предмету одежды в низком разрешении (28 на 28 пикселей):

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Рис. 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Образцы Fashion-MNIST</a> (Zalando, лицензия MIT).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST составлялся как замена классическому [MNIST](http://yann.lecun.com/exdb/mnist/) датасету, который часто используют на первых уроках в программах по компьютерному зрению - своебразный "Hello, World" в мире машинного обучения. Классический датасет MNIST состоит из изображений рукописных чисел (0, 1, 2 и так далее) в точно таком же формате, как и предметы одежды, которыми мы будем пользоваться сегодня.

Для разнообразия сегодня мы будем ипользовать Fashion MNIST, а также потому, что он представляет большую сложность в сравнении с MNIST. Оба датасета относительно малы, и нужны для проверки правильно ли работает алгоритм или нет. Их удобно использовать как отправные точки для тестирования и отладки кода.

Чтобы обучить модель мы потренируем нейросеть на 60,000 изображений, а также загрузим еще 10,000 чтобы проверить, насколько правильно нейросеть обучилась предсказывать их класс. Ты можешь загрузить Fashion MNIST прямо в TensorFlow, просто сделай импорт и загрузи картинки:

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Загрузка датасета возвращает 4 массива NumPy:

* Массивы `train_images` и `train_labels` являются *тренировочным сетом* — данными, на которых модель будет обучаться
* Модель тестируется на *проверочном сете* - против массивов `test_images` и `test_labels`

Изображения являются 28х28 массивами NumPy, где значение пикселей варьируется от 0 до 255. В столбике *Label* (далее - классы) содержится массив целых чисел от 0 до 9. Они соответствуют классу одежды, которые изображены на картинках:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th> 
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td> 
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td> 
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td> 
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td> 
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td> 
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td> 
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td> 
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td> 
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td> 
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td> 
  </tr>
</table>

Каждому изображению соответствует всего один класс одежды. Так как *названия классов* не включены в датасет, мы сохраним их здесь, чтобы позже использовать их для вывода изображений с указанием категорий одежды:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Изучаем данные

Давай посмотрим на формат данных в датасете прежде, чем начнем тренировать модель. При помощи *shape* мы видим, что датасет состоит из 60,000 изображений размером 28 х 28 пикселей: 

In [ ]:
train_images.shape

Также мы видим, что в тренировочном сете всего 60,000 меток, по одной на каждое изображение:

In [ ]:
len(train_labels)

Каждая метка - это целое число от 0 до 9:

In [ ]:
train_labels

Тренировочный сет содержит 10,000 изображений, каждое - также 28 на 28 пикселей:

In [ ]:
test_images.shape

А в тренировочном сете - ровно 10,000 меток:

In [ ]:
len(test_labels)

## Подготовка данных к обучению модели

Все данные должны быть подготовлены и обработаны одинаково перед обучением нейросети. Если мы посмотрим на первое изображение в тренировочном сете мы увидим, что значение пикселей находится в диапазоне от о до 255:

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Мы должны определить эти значения в диапазоне от 0 до 1 прежде, чем обучать нейронную сеть. Для этого переведем тип данных изображений из целых в числа с плавающей запятой, и разделим на 255.

Важно, чтобы и тренировочный, и проверочный сет были подготовлены одинаково.

Так будет выглядеть функция чтобы подготовить все изображения:

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Выведем на экран 25 изображений из тренировочного сета и укажем класс (Label) под каждым. Таким образом мы убедимся, что наши данные в правильном формате и готовы для построения и обучения нейросети.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])

## Строим модель

Построение модели нейронной сети требует правильной конфигурации каждого слоя, и последующей компиляции.

### Создаем слои

Составными частями нейросетей являются *слои*. Они извлекают точные представления из данных, на которых их обучают. И конечно же будем надеяться что в конечном итоге они извлекут что-то полезное для наших практических задач.

Большинство моделей глубокого обучения состоят из простых слоев в цепочке. Большинство же слоев, таких как например `tf.keras.layers.Dense`, имеют параметры которые используются в процессе тренировки:

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

Первый слой в нашей нейросети - `tf.keras.layers.Flatten` (flatten, пер. "Выравнивать"), будеn трансформировать изображения из 2D-массива (28 x 28 пикселей), в 1D-массив из 28 * 28 = 784 пикселей. Представь, что в этом слое мы как будто разбираем ряды сложенных пикселей изображения и выкладываем их в ряд. Этот слой не имеет никаких параметров для обучения; он просто преобразует формат данных.

После того, как мы разложили все пиксели, нейросеть будет состоять из двух Dense (пер. "Плотный") `tf.keras.layers.Dense` слоев. Это так называемые полносвязные слои. Первый `Dense` слой состоит из 128 узлов, или нейронов. Второй и последний *softmax* слой — возвращает массив с вероятностной шкалой которая должна равняться 1. Каждый нейрон содержит оценку, которая указывает вероятность принадлежности изображения к одному из 10 классов.

### Компилируем модель

Прежде чем модель будет готова для обучения, нам нужно указать еще несколько параметров. Их необходимо добавить в блоке *compile*:

* *Loss function* (пер. "Функция потерь") — измеряет точность модели во время обучения. Мы хотим минимизировать значение этой функции, чтобы \"направить\" модель в нужном направлении

* *Optimizer* — показывает каким образом обновляется модель на основе входных данных и функции потерь

* *Metrics* — показывают шаги тренировки и тестирования модели. В нашем примере мы будем использовать *accuracy*, т.е. процент изображений, которые были классифицированы правильно

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Обучение модели

Обучение модели нейронной сети требует выполнения следующих шагов:

1. Загрузить тренировочные данные в модель, в нашем случае - массивами `train_images` и `train_labels`
2. Модель учится ассоциировать изображения с правильными классами
3. Мы запрашиваем модель предсказать классы одежды из тренировочного сета `test_images`. Мы проверяем, соответствуют ли предсказанные классы меткам из массива `test_labels`. 

Чтобы начать обучение, укажи метод `model.fit` (fit, пер. "Приспосабливать", "Вмещать") - модель начнет обучаться на тренировочных данных:

In [ ]:
model.fit(train_images, train_labels, epochs=5)

Пока модель учится, мы видим показатели потери (*loss*) и точности (*acc*). В итоге модель достигает точности приблизительно 0.88 (88%), обучаясь на тренировочном сете.

## Рассчитываем точность

Далее, сравним какую точность полученная нами модель покажет на проверчном датасете:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Точность после проверки:', test_acc)

Полученная на проверчном сете точность оказалось немного ниже, чем на тренировочном сете. Этот разрыв между точностью на тренировке и тесте является примером *overfitting* (пер. "Переобучение"). Переобучение как правило возникает когда модель машинного обучения показывает результат хуже на новых данных, чем на тех, на которой она обучалась.

## Получим предсказания

Теперь, когда модель обучена, мы можем попробовать получить предсказания классов одежды на основе новых изображений:

In [ ]:
predictions = model.predict(test_images)

Здесь полученная модель будет предсказывать класс одежды для каждого изображения в проверчном датасете. 

Давай посмотрим на первое классификацию:

In [ ]:
predictions[0]

Предсказание класса представляет из себя массив из 10 чисел. Они описывают "уверенность" (*confidence*) соответствия изображения одному из 10 разных классов одежды. При помощи `np.argmax` мы можем увидеть самую высокую уверенность:

In [ ]:
np.argmax(predictions[0])

Обученная модель думает, что на первой картинке изображен бамшмак (ankle boot), или `class_names[9]`. И мы можем проверить, правильно ли это или нет:

In [ ]:
test_labels[0]

Давай напишем вспомогательную функцию для быстрого построения диаграмм и посмотрим, как выглядят предсказания:

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Теперь посмотрим на самое первое изображение (0), предсказание класса, используя нашу новую функцию:

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)

Для сравнения посмотрим на другое изображение (12):

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)

Теперь давай отобразим несколько изображений и их предсказанные классы:

1. Правильные предсказания будут отмечены синим, неправильные - красным
2. Также мы укажем процент правильности предсказания метки изображения

Обрати внимание:

Предсказания могут быть неправильными, даже если модель уверена что все верно!

In [ ]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)


Наконец, мы используем обученную модель сделать предсказание класса на одном изображении:

In [ ]:
# Берем одну картинку из проверочного сета
img = test_images[0]

print(img.shape)

Модели `tf.keras` оптимизированы делать предсказания на *batch* (пер. "Пакетах данных"), или на множестве примеров сразу. Таким образом, даже если мы используем всего 1 картинку, нам все равно необходимо добавить ее в список:

In [ ]:
# Добавляем изображение в наш пакет данных
img = (np.expand_dims(img,0))

print(img.shape)

И предсказываем класс этого изображения:

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

Выводим на экран диаграмму:

In [ ]:
plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

Метод `model.predict` возвращает нам список списков, один для каждой картинки в пакете данных.

При помощи `np.argmax` давай оставим предсказание с максимальной вероятностью для нашей картинки:

In [ ]:
np.argmax(predictions_single[0])

И, как и в первый раз, модель предсказывает нам класс 9 - это башмак!